# All Fake Tweets (*R*)

In this notebooks, we will use the "fake_followers.csv" data file, from the dataset provided by the Fake Project, as our source data file.

## Load the data file

> First, we store the file name (*and its location*) in the string '**fileName**'.

In [5]:
fileName0 = '../datasetsFULLcsv/fakeFollowersCSV/tweets.csv'

fileNames = c('../datasetsFULLcsv/socialSpambots1csv/tweets.csv', '../datasetsFULLcsv/socialSpambots2csv/tweets.csv', '../datasetsFULLcsv/socialSpambots3csv/tweets.csv', '../datasetsFULLcsv/traditionalSpambots1csv/tweets.csv')

> Using the CSV filename previously specified in '**fileName**', we can now load the file into the _data.frame_( ) named '**fakeCSV**'.

In [4]:
fakeCSV = read.csv(fileName0)
fakeTweets <- data.frame(userID = fakeCSV$user_id, id = fakeCSV$id, text = fakeCSV$text)

for (filename in fileNames) {
    temp0 = read.csv(filename)
    #fakeCSV <- rbind(fakeCSV, temp0)
    temp <- data.frame(userID = temp0$user_id, id = temp0$id, text = temp0$text)
    fakeTweets <- rbind(fakeTweets, temp)
}

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“embedded nul(s) found in input”Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“embedded nul(s) found in input”Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“embedded nul(s) found in input”Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“embedded nul(s) found in input”Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“embedded nul(s) found in input”

From the '**fakeCSV**' _data.frame_( ), we will create a smaller, simpler *data.frame*( ) named '**fakeTweets**'.  This reduction in size and complexity of '**fakeTweets**' is due to the fact that it only contains the ID number of the tweet in our database, the ID number of the user who generated the tweet, along with the text of the tweet.  

In [ ]:
#fakeTweets <- data.frame(userID = fakeCSV$user_id, id = fakeCSV$id, text = fakeCSV$text)

Now we remove web URLS, twitter usernames, twitter hashtags, punctuation, and stand-alone numeric digits.

In [6]:
# remove web URLs
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = gsub("http[[:alnum:][:punct:]]*", "", fakeTweets$text))

# remove twitter handles (@<username>)
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = gsub("#[[:alnum:][:punct:]]*", "", fakeTweets$text))

# remove hashtags (#<hashtag name>)
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = gsub("@[[:alnum:][:punct:]]*", "", fakeTweets$text))

# remove punctuation
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = gsub('[[:punct:] ]+', ' ', fakeTweets$text))

# remove numbers
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = gsub("[0-9]", "", fakeTweets$text))

# convert to lowercase
fakeTweets <- data.frame(userID = fakeTweets$userID, id = fakeTweets$id, text = tolower(fakeTweets$text))

## TidyText the data file

> Now we must tokenize the text of each tweet using the '*tidytext*' and '*dplyr*' libraries.  First, we must import the '*tidytext*' and '*dplyr*' libraries,

In [7]:
library(dplyr)
library(tidytext)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



> Then we convert the data frame of '**fakeTweets**' to the type from the '*dplyr*' library,

In [8]:
fakeTweets <- data_frame(userID = fakeTweets$userID, id = fakeTweets$id, text = as.character(fakeTweets$text))

> so that we can finally tokenize the text from each of the tweets,

In [9]:
fakeTweetsTOKENS <- fakeTweets %>%
    unnest_tokens(word, text)

## Remove '*Stop Words*'

> Now, we will remove any stop words from the text of the tweets.  To do this, we first import the '*stop_words*' dataset from the '*tidytext*' library

In [10]:
data(stop_words)

> Now, we use the '*anti_join*( )' function from the '*dplyr*' library to remove these stop wrods.

In [11]:
fakeTweetsTOKENS <- fakeTweetsTOKENS %>%
    anti_join(stop_words)

Joining, by = "word"


In [12]:
nrcWORDS <- get_sentiments("nrc")
nrcEMOTIONS <- unique(nrcWORDS$sentiment)

In [13]:
fakeTweetsNRCsentiment <- data.frame(id = 0)
for (emotion in nrcEMOTIONS){
    fakeTweetsNRCsentiment0 <- inner_join(fakeTweetsTOKENS, filter(nrcWORDS, sentiment == emotion))
    fakeTweetsNRCsentiment <- full_join(fakeTweetsNRCsentiment, fakeTweetsNRCsentiment0)
    }
fakeTweetsNRCsentiment <- data.frame(fakeTweetsNRCsentiment[-1,])
#fakeTweetsNRCsentiment

Joining, by = "word"
Joining, by = "id"
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")
Joining, by = "word"
Joining, by = c("id", "userID", "word", "sentiment")


In [14]:
attach(fakeTweetsNRCsentiment)
fakeNRCscoredTweets <- data.frame(table(id, sentiment), realFAKEcat = "fake")
detach(fakeTweetsNRCsentiment)

In [15]:
fakeNRCscoredTweets
#nrcEMOTIONS

id,sentiment,Freq,realFAKEcat
79932896,anger,0,fake
79932963,anger,0,fake
79933142,anger,0,fake
80053480,anger,0,fake
80161319,anger,0,fake
80162827,anger,0,fake
80559322,anger,1,fake
80608082,anger,0,fake
80615798,anger,0,fake
80615987,anger,0,fake


In [16]:
table(id, sentiment)
#scoredFakeTweets <- data.frame()
#fakeNRCscoredTweets


ERROR: Error in table(id, sentiment): object 'sentiment' not found


In [ ]:
trustScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "trust")$id, trust = filter(fakeNRCscoredTweets, sentiment == "trust")$Freq)
fearScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "fear")$id, fear = filter(fakeNRCscoredTweets, sentiment == "fear")$Freq)
negScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "negative")$id, negative = filter(fakeNRCscoredTweets, sentiment == "negative")$Freq)
sadnessScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "sadness")$id, sadness = filter(fakeNRCscoredTweets, sentiment == "sadness")$Freq)
angerScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "anger")$id, anger = filter(fakeNRCscoredTweets, sentiment == "anger")$Freq)
surpriseScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "surprise")$id, surprise = filter(fakeNRCscoredTweets, sentiment == "surprise")$Freq)
posScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "positive")$id, positive = filter(fakeNRCscoredTweets, sentiment == "positive")$Freq)
disgustScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "disgust")$id, disgust = filter(fakeNRCscoredTweets, sentiment == "disgust")$Freq)
joyScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "joy")$id, joy = filter(fakeNRCscoredTweets, sentiment == "joy")$Freq)
anticipationScores <- data.frame(id = filter(fakeNRCscoredTweets, sentiment == "anticipation")$id, anticipation = filter(fakeNRCscoredTweets, sentiment == "anticipation")$Freq, realFAKEcat = filter(fakeNRCscoredTweets, sentiment == "anticipation")$realFAKEcat)

In [ ]:
nrcSCORES <- full_join(trustScores, full_join(fearScores, full_join(negScores, full_join(sadnessScores, full_join(angerScores, full_join(surpriseScores, full_join(posScores, full_join(disgustScores, full_join(joyScores, anticipationScores)))))))))

In [ ]:
#nrcSCORES